# Subset ERA5 pressure level data

This notebook provides a way to subset ERA5 pressure level analysis by conserving its vertically integrated quantities.

The subsetting approach here is computationally heavy. Thus, Dask workers are applied and handled by PBS job sequences.

In [1]:
import os
import sys
import yaml
import dask
import zarr
import numpy as np
import xarray as xr
from glob import glob

sys.path.insert(0, os.path.realpath('../libs/'))
import verif_utils as vu

In [2]:
import multiprocessing
from dask.distributed import Client
from dask_jobqueue import PBSCluster

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

## Mass conserved vertical level subset

In [4]:
# --------------------------------------------------------------------- #
# subsetting function
def integral_conserved_subset_all_vars(ds, level_p, ind_select):
    # Precompute the level differences and midpoints
    diff_level_p = np.diff(level_p)
    
    # Create a helper function to compute the integral for each column
    def integral_conserved_subset_1d(x_column):
        x_column_midpoint = 0.5 * (x_column[1:] + x_column[:-1])
        x_column_area = x_column_midpoint * diff_level_p
        
        # Allocate the output array
        out_column_a = np.empty(len(ind_select)-1)
        out_column_a.fill(np.nan)
        
        for i_ind, ind in enumerate(ind_select[:-1]):
            ind_start = ind
            ind_end = ind_select[i_ind+1]
            out_column_a[i_ind] = np.sum(x_column_area[ind_start:ind_end]) / (level_p[ind_end] - level_p[ind_start])
        
        return out_column_a
    
    # Apply the function along the 'level' dimension and specify output_sizes in dask_gufunc_kwargs
    ds_out = xr.apply_ufunc(
        integral_conserved_subset_1d, ds,
        input_core_dims=[['level']],
        output_core_dims=[['new_level']],
        vectorize=True,  # Broadcast across other dimensions
        dask='parallelized',  # Enable Dask parallelism if ds is chunked
        dask_gufunc_kwargs={
            'allow_rechunk': True,  # Allow rechunking if necessary
            'output_sizes': {'new_level': len(ind_select)-1}  # Specify the size of the new dimension
        },
        output_dtypes=[float]
    )
    
    return ds_out
# --------------------------------------------------------------------- #

In [5]:
config_name = os.path.realpath('data_config_6h.yml')

with open(config_name, 'r') as stream:
    conf = yaml.safe_load(stream)

# save to zarr
base_dir = conf['zarr_opt']['save_loc'] + 'upper_subset/' 
if not os.path.exists(base_dir):
    os.makedirs(base_dir)

level_p_select = np.array(conf['zarr_opt']['subset_level'])
level_midpoints = 0.5 * (level_p_select[1:] + level_p_select[:-1])

compress = zarr.Blosc(cname='zstd', clevel=1, shuffle=zarr.Blosc.SHUFFLE, blocksize=0)


chunk_size = conf['zarr_opt']['chunk_size_4d']

chunk_size_4d = dict(chunks=(chunk_size['time'],
                             chunk_size['level'],
                             chunk_size['latitude'],
                             chunk_size['longitude']))
dict_encoding = {}

for i_var, var in enumerate(conf['RDA']['varname_upper_air']):
    dict_encoding[var] = {'compressor': compress, **chunk_size_4d}

In [6]:
year = 2000

load_name = conf['RDA']['save_loc'] + 'upper_air/' + conf['RDA']['prefix'] + '_upper_air_{}.zarr'.format(year)
load_name_Q = conf['RDA']['save_loc'] + 'upper_air/' + conf['RDA']['prefix'] + '_Q_{}.zarr'.format(year)

**Raw version**

In [7]:
ds_plevel_base = xr.open_zarr(load_name, consolidated=True)
ds_plevel_Q = xr.open_zarr(load_name_Q, consolidated=True)

ds_plevel = xr.merge([ds_plevel_base, ds_plevel_Q])

level_p = np.array(ds_plevel['level'])
mask = np.isin(level_p, level_p_select)
ind_select = np.where(mask)[0]

ds_plevel = ds_plevel.chunk({'level': -1})
ds_plevel_subset = integral_conserved_subset_all_vars(ds_plevel, level_p, ind_select)
ds_plevel_subset = ds_plevel_subset.assign_coords(new_level=level_midpoints)
ds_plevel_subset = ds_plevel_subset.rename({'new_level': 'level'})
ds_plevel_subset = ds_plevel_subset.transpose('time', 'level', 'latitude', 'longitude')
ds_plevel_subset = ds_plevel_subset.chunk(chunk_size)

save_name = base_dir + conf['zarr_opt']['prefix'] + '_L8_{}.zarr'.format(year)
# ds_plevel_subset.to_zarr(save_name, mode="w", consolidated=True, compute=True, encoding=dict_encoding)

In [11]:
save_name

'/glade/derecho/scratch/ksha/CREDIT_data/ERA5_plevel_base/upper_subset/ERA5_subset_6h_L8_2000.zarr'

**Distributed on Dask Client** 

In [ ]:
# Check if Dask client exists and shut it down
if 'client' in locals() and isinstance(client, Client):
    client.shutdown()
    print('...shutdown existing Dask client...')
else:
    print('Dask client does not exist, bulilding ...')

# Set up the Dask cluster
project_num = 'NAML0001'

cluster = PBSCluster(
    account=project_num,
    walltime='12:00:00',
    cores=1,
    memory='70GB',
    shared_temp_directory='/glade/derecho/scratch/ksha/tmp/',
    queue='casper'
)
cluster.scale(jobs=40)
client = Client(cluster)

print(f"Cluster dashboard: {client.dashboard_link}")

ds_plevel = xr.open_zarr(load_name, consolidated=True)
level_p = np.array(ds_plevel['level'])
mask = np.isin(level_p, level_p_select)
ind_select = np.where(mask)[0]

ds_plevel = ds_plevel.chunk({'level': -1})
ds_plevel_subset = integral_conserved_subset_all_vars(ds_plevel, level_p, ind_select)
ds_plevel_subset = ds_plevel_subset.assign_coords(new_level=level_midpoints)
ds_plevel_subset = ds_plevel_subset.rename({'new_level': 'level'})
ds_plevel_subset = ds_plevel_subset.transpose('time', 'level', 'latitude', 'longitude')
ds_plevel_subset = ds_plevel_subset.chunk(chunk_size)

save_name = base_dir + conf['zarr_opt']['prefix'] + '_upper_air_{}.zarr'.format(year)
ds_plevel_subset.to_zarr(save_name, mode="w", consolidated=True, compute=True, encoding=dict_encoding)

# Shutdown Dask cluster and client
print('...shutting down Dask client and cluster...')
client.close()
cluster.close()

# Removing Dask worker files
print('...removing Dask worker files...')
fns_rm = sorted(glob('./dask-worker*'))
print(f"Found {len(fns_rm)} Dask worker files.")
for fn in fns_rm:
    if os.path.exists(fn):
        os.remove(fn)
        print(f"Removed: {fn}")
    else:
        print(f"File not found: {fn}")

print('...all done...')

Dask client does not exist yet.
Cluster dashboard: http://128.117.208.150:8787/status


### How to run above as PBS jobs

One year at a time. Start from year 1979, when it finishes, move to 1980, etc.

PBS script name: `ERA5_PP_plevel_subset_6h_${year}.sh`

Inside the PBS script, the python code above, including the dask client component, is called.

```
#!/bin/bash

# List of years to process
start_year=1979
end_year=2023

# Initialize a variable to hold the job ID of the previous job
prev_job_id=""

# Loop through each year
for year in $(seq $start_year $end_year); do
    job_script="ERA5_PP_plevel_subset_6h_${year}.sh"

    if [[ -z "$prev_job_id" ]]; then
        # Submit the first job without any dependency
        job_id=$(qsub $job_script)
    else
        # Submit the current job with a dependency on the previous job
        job_id=$(qsub -W depend=afterok:$prev_job_id $job_script)
    fi

    # Extract only the job ID from the qsub output
    prev_job_id=$(echo $job_id | cut -d '.' -f 1)

    echo "Submitted job for year $year with job ID $prev_job_id"
done
```

### Combine Q and subset

In [6]:
config_name = os.path.realpath('data_config_6h.yml')

with open(config_name, 'r') as stream:
    conf = yaml.safe_load(stream)

# save to zarr
base_dir = conf['zarr_opt']['save_loc'] + 'upper_subset/' 
if not os.path.exists(base_dir):
    os.makedirs(base_dir)

level_p_select = np.array(conf['zarr_opt']['subset_level'])
level_midpoints = 0.5 * (level_p_select[1:] + level_p_select[:-1])

In [10]:
year = 1991
load_name = conf['RDA']['save_loc'] + 'upper_air/' + conf['RDA']['prefix'] + '_Q_{}.zarr'.format(year)
ds_plevel = xr.open_zarr(load_name, consolidated=True)

In [11]:
year = 1985

load_name = conf['RDA']['save_loc'] + 'upper_air/' + conf['RDA']['prefix'] + '_Q_{}.zarr'.format(year)
# variables_levels = {}
# variables_levels['specific_total_water'] = None

In [12]:
compress = zarr.Blosc(cname='zstd', clevel=1, shuffle=zarr.Blosc.SHUFFLE, blocksize=0)

chunk_size_4d = dict(chunks=(conf['zarr_opt']['chunk_size_4d']['time'],
                             conf['zarr_opt']['chunk_size_4d']['level'],
                             conf['zarr_opt']['chunk_size_4d']['latitude'],
                             conf['zarr_opt']['chunk_size_4d']['longitude']))
dict_encoding = {}
dict_encoding['specific_total_water'] = {'compressor': compress, **chunk_size_4d}

In [14]:
ds_plevel = xr.open_zarr(load_name)

FileNotFoundError: No such file or directory: '/glade/derecho/scratch/ksha/CREDIT_data/ERA5_plevel_base/upper_air/ERA5_plevel_6h_Q_1985.zarr'

In [ ]:
ds_plevel = xr.open_zarr(load_name, consolidated=True)
#ds_plevel = vu.ds_subset_everything(ds_plevel, variables_levels)

level_p = np.array(ds_plevel['level'])
mask = np.isin(level_p, level_p_select)
ind_select = np.where(mask)[0]

ds_plevel = ds_plevel.chunk({'level': -1})
ds_plevel_subset = integral_conserved_subset_all_vars(ds_plevel, level_p, ind_select)
ds_plevel_subset = ds_plevel_subset.assign_coords(new_level=level_midpoints)
ds_plevel_subset = ds_plevel_subset.rename({'new_level': 'level'})
ds_plevel_subset = ds_plevel_subset.transpose('time', 'level', 'latitude', 'longitude')
ds_plevel_subset = ds_plevel_subset.chunk(conf['zarr_opt']['chunk_size_4d'])

save_name = base_dir + conf['zarr_opt']['prefix'] + '_Q_{}.zarr'.format(year)
ds_plevel_subset.to_zarr(save_name, mode="w", consolidated=True, compute=True, encoding=dict_encoding)

In [36]:
ds_plevel_subset

<xarray.Dataset> Size: 158GB
Dimensions:               (time: 1464, latitude: 721, longitude: 1440, level: 13)
Coordinates:
  * latitude              (latitude) float64 6kB 90.0 89.75 ... -89.75 -90.0
  * longitude             (longitude) float64 12kB 0.0 0.25 0.5 ... 359.5 359.8
  * time                  (time) datetime64[ns] 12kB 2000-01-01 ... 2000-12-3...
  * level                 (level) float64 104B 25.5 75.0 125.0 ... 887.5 962.5
Data variables:
    specific_total_water  (time, level, latitude, longitude) float64 158GB dask.array<chunksize=(1, 13, 721, 1440), meta=np.ndarray>